In [1]:
# Add autoreload using % snytax
import IPython
ip = IPython.get_ipython()
if ip is not None:
    ip.run_line_magic('load_ext', 'autoreload')
    ip.run_line_magic('autoreload', '2')

In [2]:
from matplotlib import pyplot as plt
from whar_datasets.adapters.pytorch_location_adapter import PyTorchLocationAdapter
from whar_datasets.adapters.numpy_location import NumpyLocationAdapter
from whar_datasets.support.getter import WHARDatasetID, get_whar_cfg
import rich

In [6]:
def get_dataset(id: WHARDatasetID):
    cfg = get_whar_cfg(id)
    cfg.in_memory = True
    dataset = NumpyLocationAdapter(cfg)
    rich.print(dataset.samples.shape)
    rich.print(dataset.labels.shape)
    return dataset

dataset = get_dataset(WHARDatasetID.DSADS)

Creating config hash...
Checking download...
Download exists.
Checking sessions...
Sessions exist.
Validating common format...
[########################################] | 100% Completed | 955.83 ms
Common format validated.
Checking windowing...
Loading config hash...
Windowing exists.
subject_ids: [0 1 2 3 4 5 6 7]
activity_ids: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Applying unified activity space: ['walking', 'walking_upstairs', 'walking_downstairs', 'sitting', 'standing', 'laying', 'jogging', 'running', 'cycling', 'nordic_walking', 'watching_tv', 'computer_work', 'car_driving', 'vacuum_cleaning', 'ironing', 'folding_laundry', 'house_cleaning', 'playing_soccer', 'rope_jumping', 'null', 'waist_bends_forward', 'frontal_elevation_of_arms', 'knees_bending_crouching', 'jump_front_and_back', 'standing_in_an_elevator_still', 'moving_around_in_an_elevator', 'walking_threadmill', 'running_threadmill', 'exercise_stepper', 'exercise_cross_trainer', 'cycling_on_exercise_bike

Splitting windows: 100%|██████████| 17936/17936 [00:02<00:00, 6703.67it/s]


Getting normalization parameters...
Loading config hash...
Creating normalization parameters hash...
Loading normalization parameters hash...
Loading all samples from single parquet file...


Reconstructing samples: 100%|██████████| 17936/17936 [00:01<00:00, 9058.13it/s]


(53808, 125, 9, 1)

(53808,)

In [7]:
import numpy as np
print(np.unique(dataset.labels, return_counts=True))


(array([ 0,  1,  2,  3,  4,  5, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]), array([2832, 2832, 2832, 2832, 2832, 5664, 2832, 2832, 5664, 2832, 2832,
       2832, 2832, 2832, 2832, 2832, 2832]))


In [5]:
train_dataloader, val_dataloader, test_dataloader = dataset

TypeError: cannot unpack non-iterable NumpyLocationAdapter object

In [ ]:
item = next(iter(train_dataloader))
rich.print(item[0].shape)
rich.print(item[1].shape)
rich.print(item[2].shape)
rich.print(item[3])

torch.Size([16])

torch.Size([16, 128, 9])

torch.Size([16])

[
    tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
    tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
    tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
    tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
    tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
]

In [ ]:
from whar_datasets.support.dataset_activities import build_activity_alignment
from whar_datasets.support.getter import WHARDatasetID
import numpy as np

pamap_cfg = get_whar_cfg(WHARDatasetID.PAMAP2)  # adjust path if needed
pamap_adapter = NumpyLocationAdapter(pamap_cfg)

dsads_cfg = get_whar_cfg(WHARDatasetID.DSADS)  # adjust path if needed
dsads_adapter = NumpyLocationAdapter(dsads_cfg)

# 1) Build canonical activities and per-dataset mappings
canonical, mappings = build_activity_alignment(
    [WHARDatasetID.PAMAP2, WHARDatasetID.DSADS],
    strategy="intersection",  # or "union", or pass target_classes=[...]
)
print("Canonical:", canonical)

# 2) Remap labels from each dataset
def remap(labels: np.ndarray, mapping: list[int]) -> tuple[np.ndarray, np.ndarray]:
    mapping = np.array(mapping)
    new = mapping[labels]
    keep = new >= 0
    return new[keep], keep  # keep mask drops activities not in canonical

pamap_mapping = mappings[WHARDatasetID.PAMAP2]
dsads_mapping = mappings[WHARDatasetID.DSADS]

pamap_labels_remapped, pamap_keep = remap(pamap_adapter.labels, pamap_mapping)
dsads_labels_remapped, dsads_keep = remap(dsads_adapter.labels, dsads_mapping)

# 3) Filter samples with the keep masks, then concatenate as needed
pamap_samples = pamap_adapter.samples[pamap_keep]
dsads_samples = dsads_adapter.samples[dsads_keep]

Creating config hash...
Checking download...
Dataset directory not found at './datasets/pamap2'.
Extracting ./datasets/pamap2+physical+activity+monitoring.zip to ./datasets/pamap2
Extracting ./datasets/pamap2/PAMAP2_Dataset.zip to ./datasets/pamap2/PAMAP2_Dataset
Checking sessions...
Sessions directory not found at './datasets/pamap2/cache/sessions/'.
Parsing...


Caching sessions: 100%|██████████| 199/199 [00:20<00:00,  9.55it/s]

Validating common format...
[                                        ] | 0% Completed | 93.29 us

[########################################] | 100% Completed | 4.18 ss
Common format validated.
Checking windowing...
Windows directory not found at './datasets/pamap2/cache/windows/'.
Selecting activities...
[########################################] | 100% Completed | 11.38 s


Combining windows: 100%|██████████| 44584/44584 [00:05<00:00, 8306.34it/s] 


subject_ids: [0 1 2 3 4 5 6 7 8]
activity_ids: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Postprocessing...
Loading all windows from single parquet file...


Splitting windows: 100%|██████████| 44584/44584 [00:09<00:00, 4794.29it/s]


Getting normalization parameters...
Loading config hash...
Creating normalization parameters hash...
Loading normalization parameters hash...


Combining samples: 100%|██████████| 44584/44584 [00:03<00:00, 14213.41it/s]


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device